In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy.random as npr
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from keras.optimizers import Adam
from keras_nlp.layers import PositionEmbedding

In [3]:
seed = 428

np.random.seed(seed)
tf.random.set_seed(seed)
random.seed(seed)

In [4]:
def get_masked_input_and_labels(encoded_texts, n_cat):
    # For each sentence, mask each word one-by-one

    encoded_texts_masked = []
    y_labels = []

    for encoded_text in encoded_texts:
        for i in range(len(encoded_text)):
            encoded_text_masked = np.copy(encoded_text)
            y_label = encoded_text_masked[i]
            encoded_texts_masked.append(np.delete(encoded_text_masked, i))
            y_labels.append(np.array([y_label]))

    return np.array(encoded_texts_masked), np.array(y_labels)

In [5]:
#### K = number of countries = number of capitals = number of universities = number of mascots
#### M = number of words only used by each topic
#### S = number of words used by both topics
#### L = sentence length
#### q1, q2 = probability of having 1 or 2 pairs
#### embed_dim = dimension of embeddings
#### n_sentences = number of training sentences

def train_model(K, M, S, L, q1, q2, embed_dim, n_sentences):
    
    countries = ['country_' + str(i) for i in range(K)]
    capitals = ['capital_' + str(i) for i in range(K)]
    universities = ['university_' + str(i) for i in range(K)]
    mascots = ['mascot_' + str(i) for i in range(K)]
    random_capitals = ['random_capital_' + str(i) for i in range(M)]
    random_mascots = ['random_mascot_' + str(i) for i in range(M)]
    randoms = ['random_' + str(i) for i in range(S)]

    vocabs = countries + capitals + universities + mascots + random_capitals + random_mascots + randoms
    vocab_map = {}

    for i in range(len(vocabs)):
        vocab_map[vocabs[i]] = i
        
    sentences = []
    sentences_number = []
    
    q0 = 1 - q1 - q2

    for i in range(n_sentences):

        sentence = []
        
        temp = npr.uniform()
        temp2 = npr.uniform()
        
        if temp2 <= q0:
            n_pairs = 0
        elif temp2 <= q0 + q1:
            n_pairs = 1
        else:
            n_pairs = 2
        
        if temp <= 0.5: ### country - capital
        
            pairs = np.random.choice(np.arange(K), n_pairs, replace = False)
#             pairs2 = np.random.choice(np.arange(M), n_pairs, replace = False)
            for pair in pairs:
                sentence.append(countries[pair])
                sentence.append(capitals[pair])        
#             for pair in pairs2:
#                 temp3 = npr.uniform()
#                 if temp3 <= 0.5:
#                     sentence.append(random_capitals[pair])
#                 else:
#                     sentence.append(random_mascots[pair])
            
            randoms_dup = 2 * random_capitals + 1 * randoms 
            sentence += list(np.random.choice(randoms_dup, L - 2 * n_pairs, replace = False))  
                 
        else: ### university - mascot
            
            pairs = np.random.choice(np.arange(K), n_pairs, replace = False)
#             pairs2 = np.random.choice(np.arange(M), n_pairs, replace = False)
            for pair in pairs:
                sentence.append(universities[pair])
                sentence.append(mascots[pair])        
#             for pair in pairs2:
#                 temp3 = npr.uniform()
#                 if temp3 <= 0.5:
#                     sentence.append(random_mascots[pair])
#                 else:
#                     sentence.append(random_capitals[pair])

            
            randoms_dup = 1 * randoms + 2 * random_mascots
            sentence += list(np.random.choice(randoms_dup, L - 2 * n_pairs, replace = False))  
            
        
        

        #sentence += list(np.random.choice(randoms + random_capitals + random_mascots, L - 2 * n_pairs, replace = False))  

        sentence_number = [vocab_map[i] for i in sentence]
        sentences.append(sentence)
        sentences_number.append(sentence_number)
        
    x_train = np.array(sentences_number)
    n_cat = len(vocab_map)
    x_masked_train, y_masked_labels_train = get_masked_input_and_labels(x_train, n_cat)
    
    callback = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5, restore_best_weights = True)
    inputs = layers.Input((x_masked_train.shape[1],), dtype=tf.int64)
    word_embeddings = layers.Embedding(n_cat, embed_dim, name="word_embedding")(inputs)
    encoder_output = layers.GlobalAveragePooling1D()(word_embeddings)
    mlm_output = layers.Dense(n_cat, name="mlm_cls", activation="softmax", use_bias=False)(encoder_output)
    mlm_model = keras.Model(inputs = inputs, outputs = mlm_output)
    adam = Adam()
    mlm_model.compile(loss='sparse_categorical_crossentropy', optimizer=adam)

    history = mlm_model.fit(x_masked_train, y_masked_labels_train,
                        validation_split = 0.5, callbacks = [callback], 
                        epochs=500, batch_size=128, verbose=0)
    
    return sentences, vocab_map, mlm_model

In [6]:
def get_acc_prob(K, M, S, L, q1, q2, embed_dim, n_sentences, n_samples):
    
    sentences, vocab_map, current_model = train_model(K, M, S, L, q1, q2, embed_dim, n_sentences)

    acc_capitals = []
    prob_capitals = []

    for _ in range(n_samples):
        sentence = []
        random_capitals = np.random.choice(np.arange(K), int(L/2), replace = False)
        for random_capital in random_capitals:
            sentence.append('country_' + str(random_capital))
            sentence.append('capital_' + str(random_capital))
        sentence = sentence[:-1]
        sentence_number = [vocab_map[i] for i in sentence]
        temp = keras.backend.function(inputs = current_model.layers[0].input, outputs = current_model.layers[-1].output) \
            (np.array(sentence_number).reshape(1,len(sentence_number)))
        actual = vocab_map['capital_' + str(random_capitals[-1])]
        acc_capitals.append(1 if np.argsort(-1 * temp)[0][0] == actual else 0)
        prob_capitals.append(temp[0][vocab_map['capital_' + str(random_capitals[-1])]])
        
    acc_mascots = []
    prob_mascots = []

    for _ in range(n_samples):
        sentence = []
        random_mascots = np.random.choice(np.arange(K), int(L/2), replace = False)
        for random_mascot in random_mascots:
            sentence.append('university_' + str(random_mascot))
            sentence.append('mascot_' + str(random_mascot))
        sentence = sentence[:-1]
        sentence_number = [vocab_map[i] for i in sentence]
        temp = keras.backend.function(inputs = current_model.layers[0].input, outputs = current_model.layers[-1].output) \
            (np.array(sentence_number).reshape(1,len(sentence_number)))
        actual = vocab_map['mascot_' + str(random_mascots[-1])]
        acc_mascots.append(1 if np.argsort(-1 * temp)[0][0] == actual else 0)
        prob_mascots.append(temp[0][vocab_map['mascot_' + str(random_mascots[-1])]])
        

    return sentences, current_model, vocab_map, (np.mean(acc_capitals), np.mean(prob_capitals)), \
                (np.mean(acc_mascots), np.mean(prob_mascots))

In [7]:
K = 10 # number of countries
L = 8 # sentence length
M = 20 # number of words used by each topic
S = 20 # number of words used by both topics
embed_dim = 10 # CBOW embedding dimension
n_sentences = 50000 # number of sentences in the training set
n_samples = 1000

In [9]:
q0 = 0 # probability of having 0 pairs
q1 = 1 # probability of having 1 pair
q2 = 0 # probability of having 2 pairs

accs_c = 0
probs_c = 0
accs_d = 0
probs_d = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c, acc_d \
        = get_acc_prob(K, M, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    print(acc_d)
    
    accs_c += acc_c[0]/10
    probs_c += acc_c[1]/10
    accs_d += acc_d[0]/10
    probs_d += acc_d[1]/10
    
print((accs_c, probs_c))
print((accs_d, probs_d))

(0.0, 9.4843934e-20)
(0.0, 7.391937e-24)
(0.0, 1.5831181e-24)
(0.0, 1.7545277e-25)
(0.0, 2.129254e-28)
(0.0, 2.2712331e-23)
(0.0, 4.1619966e-29)
(0.0, 1.277769e-32)
(0.0, 1.979828e-31)
(0.0, 6.532465e-33)
(0.0, 7.7733754e-23)
(0.0, 1.7187979e-26)
(0.0, 2.2486198e-21)
(0.0, 1.296941e-21)
(0.0, 3.6143344e-25)
(0.0, 3.0740615e-28)
(0.0, 1.474908e-17)
(0.0, 1.552337e-20)
(0.0, 3.0998773e-27)
(0.0, 1.1886412e-25)
(0.0, 1.48462525440465e-18)
(0.0, 1.6850727718030644e-21)


In [10]:
q0 = 0 # probability of having 0 pairs
q1 = 0 # probability of having 1 pair
q2 = 1 # probability of having 2 pairs

accs_c = 0
probs_c = 0
accs_d = 0
probs_d = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c, acc_d \
        = get_acc_prob(K, M, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    print(acc_d)
    
    accs_c += acc_c[0]/10
    probs_c += acc_c[1]/10
    accs_d += acc_d[0]/10
    probs_d += acc_d[1]/10
    
print((accs_c, probs_c))
print((accs_d, probs_d))

(0.018, 0.005220848)
(0.285, 0.14097655)
(0.0, 1.9688644e-06)
(0.0, 0.000104568826)
(0.216, 0.124661334)
(0.458, 0.12060691)
(0.552, 0.442404)
(0.303, 0.21125442)
(0.104, 0.018935692)
(0.102, 0.038279306)
(0.041, 0.0074908896)
(0.384, 0.2819406)
(0.231, 0.14097819)
(0.864, 0.6389719)
(0.101, 0.08109608)
(0.464, 0.2576689)
(0.201, 0.08724061)
(0.0, 0.000497123)
(0.668, 0.44366363)
(0.045, 0.014173864)
(0.2132, 0.1351693237714471)
(0.29050000000000004, 0.17044741815625458)


In [11]:
q0 = 1/2 # probability of having 0 pairs
q1 = 1/2 # probability of having 1 pair
q2 = 0 # probability of having 2 pairs

accs_c = 0
probs_c = 0
accs_d = 0
probs_d = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c, acc_d \
        = get_acc_prob(K, M, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    print(acc_d)
    
    accs_c += acc_c[0]/10
    probs_c += acc_c[1]/10
    accs_d += acc_d[0]/10
    probs_d += acc_d[1]/10
    
print((accs_c, probs_c))
print((accs_d, probs_d))

(0.861, 0.7324312)
(0.972, 0.9196863)
(0.673, 0.5443453)
(0.781, 0.6356478)
(0.747, 0.61269724)
(0.931, 0.88556826)
(0.929, 0.84002227)
(0.822, 0.6537019)
(0.732, 0.5804807)
(0.894, 0.7792309)
(0.898, 0.77600896)
(0.874, 0.760046)
(0.884, 0.7148742)
(0.897, 0.7654695)
(0.951, 0.8445935)
(0.861, 0.7681012)
(0.87, 0.74652475)
(0.782, 0.6883448)
(0.746, 0.61168283)
(0.666, 0.5102447)
(0.8290999999999998, 0.700366097688675)
(0.8480000000000001, 0.7366041362285614)


In [12]:
q0 = 1/2 # probability of having 0 pairs
q1 = 0 # probability of having 1 pair
q2 = 1/2 # probability of having 2 pairs

accs_c = 0
probs_c = 0
accs_d = 0
probs_d = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c, acc_d \
        = get_acc_prob(K, M, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    print(acc_d)
    
    accs_c += acc_c[0]/10
    probs_c += acc_c[1]/10
    accs_d += acc_d[0]/10
    probs_d += acc_d[1]/10
    
print((accs_c, probs_c))
print((accs_d, probs_d))

(1.0, 0.9999887)
(1.0, 0.9999605)
(1.0, 0.9999664)
(1.0, 0.99992263)
(1.0, 0.99999297)
(1.0, 0.9999955)
(1.0, 0.99996644)
(1.0, 0.9999646)
(1.0, 0.9999697)
(1.0, 0.99998504)
(1.0, 0.9999807)
(1.0, 0.9999938)
(1.0, 0.9999858)
(1.0, 0.9999899)
(1.0, 0.99995166)
(1.0, 0.9999684)
(1.0, 0.9999877)
(1.0, 0.9999853)
(1.0, 0.9999843)
(1.0, 0.99997914)
(0.9999999999999999, 0.9999774396419525)
(0.9999999999999999, 0.9999744832515716)


In [13]:
q0 = 0 # probability of having 0 pairs
q1 = 1/2 # probability of having 1 pair
q2 = 1/2 # probability of having 2 pairs

accs_c = 0
probs_c = 0
accs_d = 0
probs_d = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c, acc_d \
        = get_acc_prob(K, M, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    print(acc_d)
    
    accs_c += acc_c[0]/10
    probs_c += acc_c[1]/10
    accs_d += acc_d[0]/10
    probs_d += acc_d[1]/10
    
print((accs_c, probs_c))
print((accs_d, probs_d))

(1.0, 0.9673855)
(1.0, 0.97684157)
(1.0, 0.9989292)
(1.0, 0.9940557)
(1.0, 0.9869843)
(1.0, 0.98421997)
(1.0, 0.98902285)
(1.0, 0.98814803)
(1.0, 0.9013169)
(1.0, 0.97750264)
(1.0, 0.99054736)
(1.0, 0.9785773)
(1.0, 0.9887659)
(1.0, 0.98320884)
(1.0, 0.97543544)
(1.0, 0.99167097)
(1.0, 0.989751)
(1.0, 0.9898122)
(1.0, 0.9846409)
(1.0, 0.9858713)
(0.9999999999999999, 0.9772779285907746)
(0.9999999999999999, 0.9849908530712128)


In [14]:
q0 = 1/3 # probability of having 0 pairs
q1 = 1/3 # probability of having 1 pair
q2 = 1/3 # probability of having 2 pairs

accs_c = 0
probs_c = 0
accs_d = 0
probs_d = 0

for _ in range(10):
    sentences, mlm_model, vocab_map, acc_c, acc_d \
        = get_acc_prob(K, M, S, L, q1, q2, embed_dim, n_sentences, n_samples)
    
    print(acc_c)
    print(acc_d)
    
    accs_c += acc_c[0]/10
    probs_c += acc_c[1]/10
    accs_d += acc_d[0]/10
    probs_d += acc_d[1]/10
    
print((accs_c, probs_c))
print((accs_d, probs_d))

(1.0, 0.99995416)
(1.0, 0.99996215)
(1.0, 0.99995446)
(1.0, 0.9999765)
(1.0, 0.99996215)
(1.0, 0.99996096)
(1.0, 0.9999852)
(1.0, 0.9999758)
(1.0, 0.9999793)
(1.0, 0.999974)
(1.0, 0.9999741)
(1.0, 0.9999584)
(1.0, 0.99997354)
(1.0, 0.99994075)
(1.0, 0.9999497)
(1.0, 0.9999462)
(1.0, 0.9999919)
(1.0, 0.999905)
(1.0, 0.99986833)
(1.0, 0.999932)
(0.9999999999999999, 0.9999592840671537)
(0.9999999999999999, 0.9999531745910645)
